In [1]:
from __future__ import division

import pandas as pd
import numpy as np
#import html2text
from datetime import date
import random
import os

#Packages for pulling text data 
from urllib.request import urlopen  # the lib that handles the url stuff
from bs4 import BeautifulSoup
import pandas.io.data as web
from pandas.tseries.offsets import BDay

#Packages for text data processing
import nltk, re, pprint
#nltk.download()
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer

#Packages for statistical learning 
from sklearn.preprocessing import normalize as Normal
from sklearn import svm
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from sklearn import linear_model

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\ziyi\Anaconda3\lib\site-packages\pandas\io\data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [2]:
import Text_Analytics

In [3]:
#Setup working directory 
os.chdir("C:/Users/ziyi/Desktop/Inde project/Independent Study Anseri-20170120T004329Z/Independent Study Anseri")

In [4]:
MID_index=pd.read_excel("8K data/MID Index.xlsx")
MID_index.Ticker=MID_index.Ticker.apply(lambda x:x.split(" ")[0])
Tik_Name=MID_index[["Ticker","Name"]]
Tik_Name.Name=Tik_Name.Name.apply(lambda x:x.upper())

C:\Users\ziyi\Anaconda3\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [5]:
df=pd.read_csv('8K data/Financials_list_files.csv')
#df2=df[df['Form Type'].apply(lambda x:x in ['4','8-K','SC 13G/A','10-K','10-Q'])]
df2=df[df['Form Type']=='8-K']
df2['Text']=pd.Series(index=df2.index)
df2.shape

(2180, 7)

In [ ]:
counter=0
for x, row in df2.iterrows():
    try:
        data = urlopen('http://www.sec.gov/Archives/'+row['Filename']).read()
        soup=BeautifulSoup(data, 'html.parser')
        if(row['Form Type']=='4'):
            df2.loc[x,'Text']=soup.footnotes.get_text()
        elif(row['Form Type']=='8-K'):
            text=soup.body.get_text()
            beg=text.upper().find('ITEM',1)
            end=text.upper().find('SIGNATURE',1)
            df2.loc[x,'Text']=text[beg:end]
        elif(row['Form Type']=='SC 13G/A'):
            df2.loc[x,'Text']==soup.document.get_text()
        elif(row['Form Type']=='10-K' or row['Form Type']=='10-Q'):
            text=soup.body.get_text()
            beg=text.upper().find('MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS',10000)
            end=text.upper().find('QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK',10000)
            df2.loc[x,'Text']=text[beg:end]
        else:
            continue    

    except:
        pass
    
    if (counter%100==0):
        print (counter, df2.shape[0]) 
    counter+=1

0 2180
100 2180
200 2180
300 2180
400 2180
500 2180
600 2180


In [103]:
index=df2.index

df2.Text=df2.Text.astype(str)
df2.Text=df2.Text.apply(lambda x:x.upper())
df2.Text=df2.Text.apply(lambda x:x.replace('\xa0',' '))

#Add Ticker column
df2=df2.merge(Tik_Name,how="left",left_on="Company Name",right_on="Name")
df2.index=index
df2=df2.drop(["Name"],axis=1)


C:\Users\ziyi\Anaconda3\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [155]:
#Drop Companies without price information
to_drop=["VERSUM MATERIALS INC","KEYSIGHT TECHNOLOGIES INC",'CDK GLOBAL INC',
         'KNOWLES CORP','SCIENCE APPLICATIONS INTERNATIONAL CORP']

to_drop_index=[]
for i,row in df2.iterrows():
    if row["Company Name"] in to_drop:
        to_drop_index.append(i)
to_drop_index

[7209, 8325, 8326, 8327, 8328, 9217, 9218, 9219, 9220, 9221, 9222]

In [157]:
df2=df2.drop(to_drop_index)

In [159]:
df2.to_excel('8K data/8K_Info.xlsx')